In [6]:
%pip install opencv-python tensorflow ipywidgets
%pip install tensorflow-hub

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
import tensorflow as tf
import numpy as np
import cv2
import tensorflow_hub as hub

# Load draw function dari tf-models (kalo belum ada bisa diganti)
from matplotlib import pyplot as plt
from matplotlib import patches
from matplotlib.collections import LineCollection

# Kalau draw_prediction_on_image belum ada, nanti aku buatin versinya

# Ganti model_name sesuai yang mau kau pakai
model_name = "movenet_thunder"

if "tflite" in model_name:
    raise ValueError("Realtime webcam detection cuma support format SavedModel, bukan TFLite")

# Load model dari TFHub
if "movenet_lightning" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
    input_size = 192
elif "movenet_thunder" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    input_size = 256
else:
    raise ValueError("Unsupported model name: %s" % model_name)

# Deteksi pose
def movenet(input_image):
    model = module.signatures['serving_default']
    input_image = tf.cast(input_image, dtype=tf.int32)
    outputs = model(input_image)
    keypoints_with_scores = outputs['output_0'].numpy()
    return keypoints_with_scores


In [13]:
def draw_keypoints(frame, keypoints, confidence_threshold=0.3):
    h, w, _ = frame.shape
    for kp in keypoints[0, 0]:
        y, x, c = kp
        if c > confidence_threshold:
            cx, cy = int(x * w), int(y * h)
            cv2.circle(frame, (cx, cy), 4, (0, 255, 0), -1)


In [14]:
def run_movenet_webcam():
    cap = cv2.VideoCapture("http://192.168.0.101:8080/video")  # 0: default webcam

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Gagal ambil frame.")
            break

        # Convert ke RGB & resize untuk model
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_tensor = tf.convert_to_tensor(img_rgb)
        img_tensor = tf.expand_dims(img_tensor, axis=0)
        img_tensor = tf.image.resize_with_pad(img_tensor, input_size, input_size)

        # Jalankan model
        keypoints = movenet(img_tensor)

        # Gambar keypoints ke frame asli
        draw_keypoints(frame, keypoints)

        # Tampilkan hasilnya
        cv2.imshow("MoveNet Realtime", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [15]:
run_movenet_webcam()

KeyboardInterrupt: 